In [20]:
import time
from flask import Flask, request, jsonify
from collections import namedtuple
import pandas as pd
import json
from pytrends.request import TrendReq
import requests
from statistics import mean
import sys
from tqdm import tqdm
import sys
import geopy.distance
import numpy as np

### nxn+1 distance+popularity matrix


In [2]:
pytrend = TrendReq()
startTime = time.time()

In [3]:
cities = pd.read_csv('cities_10k.csv')

In [4]:
cities.head()

,Unnamed: 0,Unnamed: 0.1,Woonplaatsen,Provincie,Landsdeel,Gemeente,Lattitude,Longitude,Population,Coordinates
0,10,12,Assen,Drenthe,Noord-Nederland,Assen,52.983333,6.550000,68798,"52° 59′ NB, 6° 33′ OL"
1,15,17,Beilen,Drenthe,Noord-Nederland,Midden-Drenthe,52.850000,6.516667,11235,"52° 51′ NB, 6° 31′ OL"
2,26,29,Coevorden,Drenthe,Noord-Nederland,Coevorden,52.650000,6.733333,35303,"52° 39′ NB, 6° 44′ OL"
3,61,64,Emmen,Drenthe,Noord-Nederland,Emmen,52.783333,6.900000,56545,"52° 47′ NB, 6° 54′ OL"
4,94,98,Hoogeveen,Drenthe,Noord-Nederland,Hoogeveen,52.716667,6.466667,40190,"52° 43′ NB, 6° 28′ OL"


In [5]:
city_names = cities['Woonplaatsen'].unique()
len_city_names = len(city_names)
popularity = [0] * len_city_names

In [6]:
new_df = pd.DataFrame(list(zip(city_names, popularity)), columns = ['City', 'Popularity']) 

In [7]:
new_df.head()

,City,Popularity
0,Assen,0
1,Beilen,0
2,Coevorden,0
3,Emmen,0
4,Hoogeveen,0


In [8]:
new_df['City'] = city_names

In [9]:
new_df['Popularity'] = popularity

In [10]:
new_df.head()

,City,Popularity
0,Assen,0
1,Beilen,0
2,Coevorden,0
3,Emmen,0
4,Hoogeveen,0


In [19]:
for index, row in new_df.iterrows():
    # print(row['c1'], row['c2'])
    popularity = pytrend.build_payload([row['City']], cat=0, timeframe='today 10-m', geo='', gprop='')
    interest_df = pytrend.interest_over_time()
    interest = mean(interest_df[row['City']])
    print(row['City'] + ': ' + str(interest))
    new_df.loc[index, 'Popularity'] = interest

ResponseError: The request failed: Google returned a response with code 400.

In [22]:
population = []
for index, row in cities.iterrows():
    population.append(row['Population'])
print(population)
norm = np.linalg.norm(population)
normal_population = population/norm
for index, row in new_df.iterrows():    
    new_df.loc[index, 'Popularity'] = normal_population[index]

[68798, 11235, 35303, 56545, 40190, 12115, 34125, 14650, 10270, 213660, 41667, 26055, 79284, 21135, 22680, 10085, 12600, 45170, 12775, 14755, 29170, 13070, 92760, 10375, 33285, 12950, 17265, 137085, 161961, 34300, 12290, 15080, 26054, 29020, 17010, 13795, 11028, 45025, 18973, 22205, 74280, 11265, 12100, 22160, 15570, 26858, 10855, 16725, 48623, 12241, 11520, 19285, 11175, 12975, 13575, 13580, 11330, 29525, 176669, 20125, 11300, 24198, 10000, 41962, 13470, 10400, 12680, 18085, 10475, 39134, 15013, 12280, 36350, 23570, 29225, 25335, 39030, 11523, 24609, 202285, 17640, 21900, 11260, 19170, 21560, 17850, 27709, 31625, 86915, 18880, 13010, 40230, 37267, 118636, 10335, 11040, 41680, 37335, 10880, 15250, 10505, 68205, 28353, 44650, 13070, 10390, 10915, 52440, 10285, 11835, 30073, 10465, 12390, 25930, 150520, 17940, 24965, 23765, 18350, 10195, 233983, 43896, 28810, 16670, 20895, 12450, 13445, 92420, 110790, 16550, 12450, 10785, 23186, 13200, 19835, 50200, 58090, 12505, 11850, 12515, 16805, 670

In [23]:
new_df.head()

,City,Popularity,Assen,Beilen,Coevorden,Emmen,Hoogeveen,Klazienaveen,Meppel,Roden,...,Spijkenisse,Vlaardingen,Voorburg,Voorhout,Voorschoten,Waddinxveen,Wassenaar,Wateringen,Zoetermeer,Zwijndrecht
0,Assen,0.045040,0.000000,15.006417,39.099434,32.411371,30.201962,40.322785,40.057228,18.434886,...,198.663529,190.930448,179.887670,163.961744,172.951455,167.652782,173.722723,187.775968,173.635066,183.229742
1,Beilen,0.007355,15.006417,0.000000,26.634321,26.888510,15.216325,34.053250,28.011894,32.023759,...,187.814627,180.405385,170.154957,154.928295,163.363364,157.024804,164.478370,177.951449,163.462857,171.489586
2,Coevorden,0.023112,39.099434,26.634321,0.000000,18.627856,19.500270,19.280238,37.615833,57.452282,...,188.421695,181.956344,173.801573,160.438243,167.464948,158.714154,169.341606,181.256606,166.173632,169.863645
3,Emmen,0.037018,32.411371,26.888510,18.627856,0.000000,30.186348,7.912676,49.281842,50.005770,...,205.456065,198.711003,189.875018,175.802466,183.364425,175.368041,184.974262,197.463129,182.533993,187.414119
4,Hoogeveen,0.026311,30.201962,15.216325,19.500270,30.186348,0.000000,34.956723,19.239744,46.422848,...,176.614971,169.572932,160.208391,145.815726,153.605408,146.179685,155.083068,167.872296,153.077424,159.398522


In [24]:
all_jobs = len_city_names * len_city_names
count = 1
toolbar_width = 40
for col in city_names:
    column = []
    start_time = time.time()
    for row in city_names:     
        c1_la = float(cities.loc[cities['Woonplaatsen'] == col, 'Lattitude'])
        c1_lo = float(cities.loc[cities['Woonplaatsen'] == col, 'Longitude'])
        c2_la = float(cities.loc[cities['Woonplaatsen'] == row, 'Lattitude'])
        c2_lo = float(cities.loc[cities['Woonplaatsen'] == row, 'Longitude'])
        coords_1 = (c1_la, c1_lo)
        #print(float(c1_la))
        #print(c1_lo)
        coords_2 = (c2_la, c2_lo)
        #print(coords_2)
        distance = geopy.distance.distance(coords_1, coords_2).km
        column.append(distance)
        division = count/all_jobs
        #print (str(count) + '/' + str(all_jobs))
        count+=1
        #print(column)
    end_time = time.time()
    print("total time taken this loop: ", end_time - start_time)
    new_df[col] = column
    print(new_df.head())

total time taken this loop:  0.6472735404968262
        City  Popularity      Assen     Beilen  Coevorden      Emmen  \
0      Assen    0.045040   0.000000  15.006417  39.099434  32.411371   
1     Beilen    0.007355  15.006417   0.000000  26.634321  26.888510   
2  Coevorden    0.023112  39.099434  26.634321   0.000000  18.627856   
3      Emmen    0.037018  32.411371  26.888510  18.627856   0.000000   
4  Hoogeveen    0.026311  30.201962  15.216325  19.500270  30.186348   

   Hoogeveen  Klazienaveen     Meppel      Roden  ...  Spijkenisse  \
0  30.201962     40.322785  40.057228  18.434886  ...   198.663529   
1  15.216325     34.053250  28.011894  32.023759  ...   187.814627   
2  19.500270     19.280238  37.615833  57.452282  ...   188.421695   
3  30.186348      7.912676  49.281842  50.005770  ...   205.456065   
4   0.000000     34.956723  19.239744  46.422848  ...   176.614971   

   Vlaardingen    Voorburg    Voorhout  Voorschoten  Waddinxveen   Wassenaar  \
0   190.930448  17

[5 rows x 329 columns]
total time taken this loop:  0.6043825149536133
        City  Popularity      Assen     Beilen  Coevorden      Emmen  \
0      Assen    0.045040   0.000000  15.006417  39.099434  32.411371   
1     Beilen    0.007355  15.006417   0.000000  26.634321  26.888510   
2  Coevorden    0.023112  39.099434  26.634321   0.000000  18.627856   
3      Emmen    0.037018  32.411371  26.888510  18.627856   0.000000   
4  Hoogeveen    0.026311  30.201962  15.216325  19.500270  30.186348   

   Hoogeveen  Klazienaveen     Meppel      Roden  ...  Spijkenisse  \
0  30.201962     40.322785  40.057228  18.434886  ...   198.663529   
1  15.216325     34.053250  28.011894  32.023759  ...   187.814627   
2  19.500270     19.280238  37.615833  57.452282  ...   188.421695   
3  30.186348      7.912676  49.281842  50.005770  ...   205.456065   
4   0.000000     34.956723  19.239744  46.422848  ...   176.614971   

   Vlaardingen    Voorburg    Voorhout  Voorschoten  Waddinxveen   Wassenaa

[5 rows x 329 columns]
total time taken this loop:  0.5146253108978271
        City  Popularity      Assen     Beilen  Coevorden      Emmen  \
0      Assen    0.045040   0.000000  15.006417  39.099434  32.411371   
1     Beilen    0.007355  15.006417   0.000000  26.634321  26.888510   
2  Coevorden    0.023112  39.099434  26.634321   0.000000  18.627856   
3      Emmen    0.037018  32.411371  26.888510  18.627856   0.000000   
4  Hoogeveen    0.026311  30.201962  15.216325  19.500270  30.186348   

   Hoogeveen  Klazienaveen     Meppel      Roden  ...  Spijkenisse  \
0  30.201962     40.322785  40.057228  18.434886  ...   198.663529   
1  15.216325     34.053250  28.011894  32.023759  ...   187.814627   
2  19.500270     19.280238  37.615833  57.452282  ...   188.421695   
3  30.186348      7.912676  49.281842  50.005770  ...   205.456065   
4   0.000000     34.956723  19.239744  46.422848  ...   176.614971   

   Vlaardingen    Voorburg    Voorhout  Voorschoten  Waddinxveen   Wassenaa

[5 rows x 329 columns]
total time taken this loop:  0.5066440105438232
        City  Popularity      Assen     Beilen  Coevorden      Emmen  \
0      Assen    0.045040   0.000000  15.006417  39.099434  32.411371   
1     Beilen    0.007355  15.006417   0.000000  26.634321  26.888510   
2  Coevorden    0.023112  39.099434  26.634321   0.000000  18.627856   
3      Emmen    0.037018  32.411371  26.888510  18.627856   0.000000   
4  Hoogeveen    0.026311  30.201962  15.216325  19.500270  30.186348   

   Hoogeveen  Klazienaveen     Meppel      Roden  ...  Spijkenisse  \
0  30.201962     40.322785  40.057228  18.434886  ...   198.663529   
1  15.216325     34.053250  28.011894  32.023759  ...   187.814627   
2  19.500270     19.280238  37.615833  57.452282  ...   188.421695   
3  30.186348      7.912676  49.281842  50.005770  ...   205.456065   
4   0.000000     34.956723  19.239744  46.422848  ...   176.614971   

   Vlaardingen    Voorburg    Voorhout  Voorschoten  Waddinxveen   Wassenaa

[5 rows x 329 columns]
total time taken this loop:  0.5405526161193848
        City  Popularity      Assen     Beilen  Coevorden      Emmen  \
0      Assen    0.045040   0.000000  15.006417  39.099434  32.411371   
1     Beilen    0.007355  15.006417   0.000000  26.634321  26.888510   
2  Coevorden    0.023112  39.099434  26.634321   0.000000  18.627856   
3      Emmen    0.037018  32.411371  26.888510  18.627856   0.000000   
4  Hoogeveen    0.026311  30.201962  15.216325  19.500270  30.186348   

   Hoogeveen  Klazienaveen     Meppel      Roden  ...  Spijkenisse  \
0  30.201962     40.322785  40.057228  18.434886  ...   198.663529   
1  15.216325     34.053250  28.011894  32.023759  ...   187.814627   
2  19.500270     19.280238  37.615833  57.452282  ...   188.421695   
3  30.186348      7.912676  49.281842  50.005770  ...   205.456065   
4   0.000000     34.956723  19.239744  46.422848  ...   176.614971   

   Vlaardingen    Voorburg    Voorhout  Voorschoten  Waddinxveen   Wassenaa

...

[5 rows x 329 columns]
total time taken this loop:  0.6126704216003418
        City  Popularity      Assen     Beilen  Coevorden      Emmen  \
0      Assen    0.045040   0.000000  15.006417  39.099434  32.411371   
1     Beilen    0.007355  15.006417   0.000000  26.634321  26.888510   
2  Coevorden    0.023112  39.099434  26.634321   0.000000  18.627856   
3      Emmen    0.037018  32.411371  26.888510  18.627856   0.000000   
4  Hoogeveen    0.026311  30.201962  15.216325  19.500270  30.186348   

   Hoogeveen  Klazienaveen     Meppel      Roden  ...  Spijkenisse  \
0  30.201962     40.322785  40.057228  18.434886  ...   198.663529   
1  15.216325     34.053250  28.011894  32.023759  ...   187.814627   
2  19.500270     19.280238  37.615833  57.452282  ...   188.421695   
3  30.186348      7.912676  49.281842  50.005770  ...   205.456065   
4   0.000000     34.956723  19.239744  46.422848  ...   176.614971   

   Vlaardingen    Voorburg    Voorhout  Voorschoten  Waddinxveen   Wassenaa

[5 rows x 329 columns]
total time taken this loop:  0.6178710460662842
        City  Popularity      Assen     Beilen  Coevorden      Emmen  \
0      Assen    0.045040   0.000000  15.006417  39.099434  32.411371   
1     Beilen    0.007355  15.006417   0.000000  26.634321  26.888510   
2  Coevorden    0.023112  39.099434  26.634321   0.000000  18.627856   
3      Emmen    0.037018  32.411371  26.888510  18.627856   0.000000   
4  Hoogeveen    0.026311  30.201962  15.216325  19.500270  30.186348   

   Hoogeveen  Klazienaveen     Meppel      Roden  ...  Spijkenisse  \
0  30.201962     40.322785  40.057228  18.434886  ...   198.663529   
1  15.216325     34.053250  28.011894  32.023759  ...   187.814627   
2  19.500270     19.280238  37.615833  57.452282  ...   188.421695   
3  30.186348      7.912676  49.281842  50.005770  ...   205.456065   
4   0.000000     34.956723  19.239744  46.422848  ...   176.614971   

   Vlaardingen    Voorburg    Voorhout  Voorschoten  Waddinxveen   Wassenaa

[5 rows x 329 columns]
total time taken this loop:  0.5245985984802246
        City  Popularity      Assen     Beilen  Coevorden      Emmen  \
0      Assen    0.045040   0.000000  15.006417  39.099434  32.411371   
1     Beilen    0.007355  15.006417   0.000000  26.634321  26.888510   
2  Coevorden    0.023112  39.099434  26.634321   0.000000  18.627856   
3      Emmen    0.037018  32.411371  26.888510  18.627856   0.000000   
4  Hoogeveen    0.026311  30.201962  15.216325  19.500270  30.186348   

   Hoogeveen  Klazienaveen     Meppel      Roden  ...  Spijkenisse  \
0  30.201962     40.322785  40.057228  18.434886  ...   198.663529   
1  15.216325     34.053250  28.011894  32.023759  ...   187.814627   
2  19.500270     19.280238  37.615833  57.452282  ...   188.421695   
3  30.186348      7.912676  49.281842  50.005770  ...   205.456065   
4   0.000000     34.956723  19.239744  46.422848  ...   176.614971   

   Vlaardingen    Voorburg    Voorhout  Voorschoten  Waddinxveen   Wassenaa

[5 rows x 329 columns]
total time taken this loop:  0.5734641551971436
        City  Popularity      Assen     Beilen  Coevorden      Emmen  \
0      Assen    0.045040   0.000000  15.006417  39.099434  32.411371   
1     Beilen    0.007355  15.006417   0.000000  26.634321  26.888510   
2  Coevorden    0.023112  39.099434  26.634321   0.000000  18.627856   
3      Emmen    0.037018  32.411371  26.888510  18.627856   0.000000   
4  Hoogeveen    0.026311  30.201962  15.216325  19.500270  30.186348   

   Hoogeveen  Klazienaveen     Meppel      Roden  ...  Spijkenisse  \
0  30.201962     40.322785  40.057228  18.434886  ...   198.663529   
1  15.216325     34.053250  28.011894  32.023759  ...   187.814627   
2  19.500270     19.280238  37.615833  57.452282  ...   188.421695   
3  30.186348      7.912676  49.281842  50.005770  ...   205.456065   
4   0.000000     34.956723  19.239744  46.422848  ...   176.614971   

   Vlaardingen    Voorburg    Voorhout  Voorschoten  Waddinxveen   Wassenaa

[5 rows x 329 columns]
total time taken this loop:  0.5365662574768066
        City  Popularity      Assen     Beilen  Coevorden      Emmen  \
0      Assen    0.045040   0.000000  15.006417  39.099434  32.411371   
1     Beilen    0.007355  15.006417   0.000000  26.634321  26.888510   
2  Coevorden    0.023112  39.099434  26.634321   0.000000  18.627856   
3      Emmen    0.037018  32.411371  26.888510  18.627856   0.000000   
4  Hoogeveen    0.026311  30.201962  15.216325  19.500270  30.186348   

   Hoogeveen  Klazienaveen     Meppel      Roden  ...  Spijkenisse  \
0  30.201962     40.322785  40.057228  18.434886  ...   198.663529   
1  15.216325     34.053250  28.011894  32.023759  ...   187.814627   
2  19.500270     19.280238  37.615833  57.452282  ...   188.421695   
3  30.186348      7.912676  49.281842  50.005770  ...   205.456065   
4   0.000000     34.956723  19.239744  46.422848  ...   176.614971   

   Vlaardingen    Voorburg    Voorhout  Voorschoten  Waddinxveen   Wassenaa

In [26]:
new_df.to_csv('city_matrix_new.csv')

In [27]:
population = new_df['Popularity']


In [28]:
population.describe()

count    327.000000
mean       0.027360
std        0.048132
min        0.006547
25%        0.008889
50%        0.015046
75%        0.025997
max        0.569368
Name: Popularity, dtype: float64